## Some examples on how to use MoreFixes dataset
It's worth to note (as mentioned in the paper) that in `fixes` tables, currently all candidate fix commits are included, so based on the requirements, you can either select only commits with score higher than specific threshold, or such candidanes by running a `delete` query.  

In [2]:
%pip install pandas matplotlib seaborn numpy requests psycopg2-binary sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
from psycopg2 import OperationalError
from pathlib import Path
from datetime import date
import numpy as np
import seaborn as sns
import matplotlib.ticker as tick
import requests
import difflib as diff
import re 
import csv
import ast
%matplotlib inline

# pd.set_option('mode.chained_assignment', None)
from sqlalchemy import create_engine

def create_connection(db_name, db_user, db_password, db_host, db_port):
    """
    create a connection to a PostgreSQL database using SQLAlchemy
    """
    try:
        engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    return engine

# Update these variables with your PostgreSQL database connection details
db_name = "postgrescvedumper"
db_user = "postgrescvedumper"
db_password = "samplepostgrespasswordpleasechange"
db_host = "127.0.0.1"
db_port = "5432"


DATA_PATH = Path.cwd().parents[0] / 'Data'
FIGURE_PATH = Path.cwd() / 'figures'
RESULT_PATH = Path.cwd() / 'results'

Path(DATA_PATH).mkdir(parents=True, exist_ok=True)
Path(FIGURE_PATH).mkdir(parents=True, exist_ok=True)
Path(RESULT_PATH).mkdir(parents=True, exist_ok=True)



conn = create_connection(db_name, db_user, db_password, db_host, db_port)

### Quering to MoreFixes database to get the parts as the separate datasets

- An example query to retrieve all the methods of C programming language. 

- Another example of query to extract all the files that have added/removed only a single statement to fix the vulnerabilities.

In [4]:
query = """
SELECT cv.cve_id, f.filename, f.num_lines_added, f.num_lines_deleted, f.code_before, f.code_after, cc.cwe_id 
FROM file_change f, commits c, fixes fx, cve cv, cwe_classification cc
WHERE f.hash = c.hash 
AND c.hash = fx.hash 
AND fx.cve_id = cv.cve_id 
AND cv.cve_id = cc.cve_id 
AND f.num_lines_added<=1 
AND f.num_lines_deleted<=1;
"""
java_single_line_fixes = pd.read_sql_query(query, conn)
java_single_line_fixes.head(5)

,cve_id,filename,num_lines_added,num_lines_deleted,code_before,code_after,cwe_id
0,CVE-2022-1575,service-worker.js,1,1,"if(!self.define){const e=e=>{""require""!==e&&(e...","if(!self.define){const e=e=>{""require""!==e&&(e...",CWE-79
1,CVE-2022-1575,VERSION,1,1,17.5.1,18.0.0,CWE-79
2,CVE-2022-1575,mxClient.js,1,1,"var mxClient={VERSION:""17.5.1"",IS_IE:null!=nav...","var mxClient={VERSION:""18.0.0"",IS_IE:null!=nav...",CWE-79
3,CVE-2022-1815,dia_he.txt,1,0,"# *DO NOT DIRECTLY EDIT THIS FILE, IT IS AUTOM...","# *DO NOT DIRECTLY EDIT THIS FILE, IT IS AUTOM...",CWE-918
4,CVE-2022-1815,dia_hi.txt,1,0,"# *DO NOT DIRECTLY EDIT THIS FILE, IT IS AUTOM...","# *DO NOT DIRECTLY EDIT THIS FILE, IT IS AUTOM...",CWE-918


## Top 10 major projects that have contributed most of the vulnerability data

In [5]:
commit_freq_query = """
SELECT fx.repo_url as commit_project, count(fx.hash) as commit_count
FROM commits c, fixes fx
WHERE fx.hash=c.hash
GROUP BY fx.repo_url
ORDER BY commit_count DESC;
"""

file_freq_query = """
SELECT fx.repo_url as file_project, count(f.file_change_id) as file_count 
FROM commits c, fixes fx, file_change f
WHERE fx.hash=c.hash
AND c.hash = f.hash
GROUP BY fx.repo_url
ORDER BY file_count DESC;
"""
method_freq_query = """
SELECT fx.repo_url as method_project, count(m.method_change_id) as method_count 
FROM commits c, fixes fx, file_change f, method_change m
WHERE fx.hash=c.hash
AND c.hash = f.hash
AND f.file_change_id = m.file_change_id
GROUP BY fx.repo_url
ORDER BY method_count DESC;
"""

df_commit_freq = pd.read_sql_query(commit_freq_query, con=conn)
df_file_freq = pd.read_sql_query(file_freq_query, con=conn)
df_method_freq = pd.read_sql_query(method_freq_query, con=conn)

- Ordering the projects on the basis of number of commits:

In [6]:
df_meta = df_commit_freq.merge(df_file_freq, 
                               left_on='commit_project', 
                               right_on='file_project').merge(df_method_freq, 
                                                             left_on='commit_project', 
                                                             right_on='method_project')
df_meta = df_meta.replace(np.nan,0)
df_meta['commit_project'] = df_meta['commit_project'].str.split('/').str[-1]
df_meta = df_meta.drop(['file_project','method_project'], axis=1)
df_meta[['commit_count','file_count', 'method_count']] = df_meta[['commit_count','file_count', 'method_count']].astype(int)
df_meta=df_meta.sort_values(by=['commit_count','file_count','method_count'], ascending=False)
df_meta.head(10)

,commit_project,commit_count,file_count,method_count
0,linux,3356,5712,13048
1,ImageMagick,937,1299,3255
2,ChakraCore,815,3582,9698
3,gpac,782,1214,4060
4,qemu,502,647,1804
5,go,435,59,123
6,radare2,400,847,2490
7,php-src,384,3,11
8,FFmpeg,355,650,2153
9,radare2,308,608,1811


### Ordering the projects on the basis of #commits, #files and #methods separately in a table. 

In [10]:
df_merged = pd.concat([df_commit_freq, df_file_freq], axis=1)
df_merged = pd.concat([df_merged, df_method_freq], axis=1)
df_merged = df_merged.replace(np.nan, 0)
df_merged['commit_project'] = df_merged['commit_project'].str.split('/').str[-1]
df_merged['file_project'] = df_merged['file_project'].str.split('/').str[-1]
df_merged['method_project'] = df_merged['method_project'].str.split('/').str[-1]

df_merged[['commit_count','file_count', 'method_count']] = df_merged[['commit_count','file_count', 'method_count']].astype(int)
df_merged = df_merged.replace('_','-', regex=True)

df_merged.head(10).to_csv(RESULT_PATH / 'vul_projects_10.csv', index=False)
df_merged.head(10)

,commit_project,commit_count,file_project,file_count,method_project,method_count
0,linux,3356,linux,5712,KodExplorer,15893
1,ImageMagick,937,magento2,4162,linux,13048
2,ChakraCore,815,ChakraCore,3582,ChakraCore,9698
3,gpac,782,Zenario,2277,suitecrm,8262
4,wireshark,738,libdwarf-code,1911,exponent-cms,5623
5,qemu,502,ImageMagick,1299,libdwarf-code,4118
6,tensorflow,475,gpac,1214,gpac,4060
7,go,435,bc-java,1208,blender,3359
8,jasper,421,FreeRTOS-Kernel,1054,ImageMagick,3255
9,exiv2,419,ILIAS,989,ChakraCore,3207


### All CVEs mapped to open source projects

In [30]:
cves_in_oss = pd.read_sql_query("""SELECT cve_id AS cve, repo_url AS url FROM fixes UNION SELECT cve, project_url AS url FROM cve_project;""", con=conn)

print(f"Total CVE mapped to opensource projects: {cves_in_oss.shape[0]}")
cves_in_oss.head(10)

Total CVE mapped to opensource projects: 80751


,cve,url
0,CVE-1999-0001,https://github.com/freebsd/freebsd
1,CVE-1999-0001,https://github.com/libressl/openbsd
2,CVE-1999-0009,https://github.com/isc-projects/bind9
3,CVE-1999-0010,https://github.com/isc-projects/bind9
4,CVE-1999-0011,https://github.com/isc-projects/bind9
5,CVE-1999-0024,https://github.com/isc-projects/bind9
6,CVE-1999-0037,https://github.com/freebsd/freebsd
7,CVE-1999-0045,https://github.com/apache/httpd
8,CVE-1999-0052,https://github.com/freebsd/freebsd
9,CVE-1999-0052,https://github.com/libressl/openbsd


### Changed files of Javascript code injection and command injection vulnerabilities

In [52]:
top_cwes_in_pls = pd.read_sql_query("""select * from file_change where hash in (select hash from fixes where cve_id in (
    (select cve_id
     from cwe_classification
     where cwe_id in ('CWE-77', 'CWE-94')
     union
     select cve_id
     from cve
     where description ilike '%code injection%' or description ilike '%code execution%' or description ilike '%command execution%' or description ilike '%command injection%'
     )
))
    and new_path not ilike '%test%'
    and (
        programming_language='JavaScript'
        or
        new_path ilike '%.js'
    )
    order by num_lines_added asc""".replace('%', '%%'), con=conn)


#select * from file_change where diff ilike '%replace(/\</g,%' and new_path not ilike '%test%'

print(f"Count of samples: {top_cwes_in_pls.shape[0]}")
top_cwes_in_pls.head(10)

Count of samples: 495


,file_change_id,hash,filename,old_path,new_path,change_type,diff,diff_parsed,num_lines_added,num_lines_deleted,code_after,code_before,nloc,complexity,token_count,programming_language
0,15865603577830,df8fcffb1959c6eb52db006508ac00a4153274cf,router.js,src/router.js,src/router.js,ModificationType.MODIFY,"@@ -19,7 +19,6 @@ import DockerHosts from ""./c...","{'added': [], 'deleted': [(22, 'import Plugins...",0,5,"import { createRouter, createWebHistory } from...","import { createRouter, createWebHistory } from...",184.0,3.0,602.0,TypeScript
1,160780018657723,504825938c0abfa2fb8ff8d529308830a5ae42ed,sanitizer.ts,packages/apputils/src/sanitizer.ts,packages/apputils/src/sanitizer.ts,ModificationType.MODIFY,"@@ -624,7 +624,6 @@ export class Sanitizer imp...","{'added': [], 'deleted': [(627, "" 'acti...",0,1,// Copyright (c) Jupyter Development Team.\n//...,// Copyright (c) Jupyter Development Team.\n//...,913,0,781,JavaScript
2,72426194346569,c12418a026f73df645ba927fd29358efe02fed1e,file-handlers.ts,src/file-handlers.ts,src/file-handlers.ts,ModificationType.MODIFY,"@@ -18,9 +18,6 @@ interface FileOptions extend...","{'added': [], 'deleted': [(21, ' settings?: {...",0,13,"// express is set like: app.engine('html', req...","// express is set like: app.engine('html', req...",60,13,361,JavaScript
3,11524814694789,79ca5608c4f77d9a5c8a4996e204377c158a6976,IDirectoryPlace.java,src/main/java/emissary/directory/IDirectoryPla...,src/main/java/emissary/directory/IDirectoryPla...,ModificationType.MODIFY,"@@ -43,21 +43,6 @@ public interface IDirectory...","{'added': [], 'deleted': [(46, ' /**'), (47...",0,29,package emissary.directory;\n\nimport java.uti...,package emissary.directory;\n\nimport java.uti...,23.0,0.0,167.0,JavaScript
4,105223913858594,8a72d8c20f38bbcac031a76a51238aa528f68821,utils.js,utils.js,utils.js,ModificationType.MODIFY,"@@ -5203,64 +5203,6 @@ FLP.next = function() {...","{'added': [], 'deleted': [(5206, 'exports.sync...",0,144,'use strict';\n\nconst Dns = require('dns');\n...,'use strict';\n\nconst Dns = require('dns');\n...,4973.0,1897.0,38564.0,JavaScript
5,206014998040691,7f1c43094e8a82e4d8f036ff7d42568fed00699d,router.js,lib/router.js,lib/router.js,ModificationType.MODIFY,"@@ -2072,10 +2072,6 @@ router.get('/pingwest/u...","{'added': [], 'deleted': [(2075, '// 篝火营地'), (...",0,14,const Router = require('@koa/router');\nconst ...,const Router = require('@koa/router');\nconst ...,1817.0,3.0,21677.0,JavaScript
6,186936832933093,ff2f6e6b91913a7be42be1b5917330fe442f2ede,index.js,dist/index.js,dist/index.js,ModificationType.MODIFY,Binary files a/dist/index.js and b/dist/index....,"{'added': [], 'deleted': []}",0,0,require('./sourcemap-register.js');/******/ ((...,require('./sourcemap-register.js');/******/ ((...,43914.0,9965.0,226464.0,JavaScript
7,195812659730532,df8fcffb1959c6eb52db006508ac00a4153274cf,server.js,server/server.js,server/server.js,ModificationType.MODIFY,"@@ -147,7 +147,6 @@ const { apiKeySocketHandle...","{'added': [], 'deleted': [(150, 'const { plugi...",0,3,"/*\n * Uptime Kuma Server\n * node ""server/ser...","/*\n * Uptime Kuma Server\n * node ""server/ser...",1316.0,178.0,8193.0,JavaScript
8,238439838373273,716b1e13042866565e00e85fd4ec490e186c4a2f,index.js,dist/index.js,dist/index.js,ModificationType.MODIFY,Binary files a/dist/index.js and b/dist/index....,"{'added': [], 'deleted': []}",0,0,require('./sourcemap-register.js');/******/ ((...,require('./sourcemap-register.js');/******/ ((...,43914.0,9965.0,226464.0,JavaScript
9,221147717847585,24c8b4368adebecec8a1d310a0f20c3b170e829e,recordfinder.js,modules/backend/formwidgets/recordfinder/asset...,modules/backend/formwidgets/recordfinder/asset...,ModificationType.MODIFY,"@@ -63,7 +63,6 @@\n RecordFinder.prototype...","{'added': [], 'deleted': [(66, '')]}",0,1,/*\n * RecordFinder plugin\n *\n * Data attrib...,/*\n * RecordFinder plugin\n *\n * Data attrib...,66.0,18.0,559.0,JavaScript
